In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from numpy.random import seed

import re
import sklearn

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [23]:
from pandas.io.json import json_normalize 
data = pd.read_csv('/Users/Sandi/Desktop/Projects/Capstone_2/questionnaire.csv')
demographics = pd.read_csv('/Users/Sandi/Desktop/Projects/Capstone_2/demographic.csv')
labs = pd.read_csv('/Users/Sandi/Desktop/Projects/Capstone_2/labs.csv')

**Data Set Merge and Select Attribute**

In [24]:
data.drop(['SEQN'], axis = 1, inplace=True)
demographics.drop(['SEQN'], axis = 1, inplace=True)
labs.drop(['SEQN'], axis = 1, inplace=True)
#https://www.kaggle.com/what0919/diabetes-prediction

In [26]:
df = pd.concat([data, demographics], axis=1, join='inner')
df = pd.concat([df, labs], axis=1, join='inner')
df.describe()
#merge

,ACD011A,ACD011B,ACD011C,ACD040,ACD110,ALQ101,ALQ110,ALQ120Q,ALQ120U,ALQ130,...,URXUTL,URDUTLLC,URXUTU,URDUTULC,URXUUR,URDUURLC,URXPREG,URXUAS,LBDB12,LBDB12SI
count,5548.0,16.0,166.0,2292.000000,975.000000,5223.000000,1576.000000,4312.000000,3460.000000,3463.000000,...,2664.000000,2664.000000,2664.000000,2664.000000,2664.000000,2664.000000,1261.000000,2662.000000,5316.000000,5316.000000
mean,1.0,8.0,9.0,3.109075,2.949744,1.312464,1.595812,4.787801,1.924566,3.534796,...,0.186950,0.007883,0.150535,0.162162,0.012038,0.177177,1.969072,15.670639,640.100263,472.394865
std,0.0,0.0,0.0,1.513612,1.731617,0.548327,0.619028,35.079943,0.854441,29.410605,...,0.133833,0.088452,0.576409,0.368669,0.034026,0.381890,0.251651,46.735936,751.967084,554.951537
min,1.0,8.0,9.0,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,0.013000,0.000000,0.013000,0.000000,0.001400,0.000000,1.000000,0.280000,18.000000,13.300000
25%,1.0,8.0,9.0,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.090000,0.000000,0.028000,0.000000,0.002600,0.000000,2.000000,3.210000,381.000000,281.200000
50%,1.0,8.0,9.0,3.000000,3.000000,1.000000,2.000000,2.000000,2.000000,2.000000,...,0.159000,0.000000,0.069000,0.000000,0.005100,0.000000,2.000000,6.240000,514.000000,379.300000
75%,1.0,8.0,9.0,4.000000,5.000000,2.000000,2.000000,4.000000,3.000000,3.000000,...,0.252000,0.000000,0.150000,0.000000,0.010700,0.000000,2.000000,12.435000,711.000000,524.700000
max,1.0,8.0,9.0,9.000000,5.000000,9.000000,9.000000,999.000000,3.000000,999.000000,...,1.290000,1.000000,25.760000,1.000000,0.843400,1.000000,3.000000,1071.300000,26801.000000,19779.100000


**NaN Handling and Feature Column Selection**

In [28]:
from sklearn.feature_selection import VarianceThreshold

df.dropna(axis=1, how='all')
df.dropna(axis=0, how='all')




df = df.rename(columns = {'SEQN' : 'ID',
                          'RIAGENDR' : 'Gender',
                          'WHD120' : 'WeightAt25Yrs', 
                          'WHD130' : 'HeightAt25Yrs',
                          'LBXPLTSI' : 'PlateletCount',
                          'LBXSCK' : 'CreatinePhosphokinase',
                          'MCQ160a' : 'HasArthritis',
                          'MCQ010' : 'HasAsthma',
                          'CSQ204' : 'AllergyCongestionPast12Months',
                          'DRABF' : 'Breast_fed',
                          'DBQ229' : 'UseMilk>=5x/wk',
                          'DBQ235B' : 'DrankMilkAge13to17yrs',
                          'DBQ235C' : 'DrankMilkAge18to35yrs',
                          'PUQ100' : 'ChemicalPesticideUsedInHomePast7days',
                          'PUQ110' : 'ChemicalWeedKillUsedExteriorHomePast7days',
                          'PAQ677' : 'Past7DaysCardio20Minutes+',
                          'PAQ678' : 'Past7DaysStrengthTone20Minutes+',
                          'PAQ706' : 'Past7DaysCardio60Minutes+'})

df = df.loc[:, ['ID', 'Gender', 'WeightAt25Yrs', 'HeightAt25Yrs','PlateletCount', 'CreatinePhosphokinase', 
                'HasArthritis', 'HasAsthma', 'AllergyCongestionPast12Months', 'Breast_fed', 'UseMilk>=5x/wk', 
                'DrankMilkAge13to17yrs', 'DrankMilkAge18to35yrs', 'ChemicalPesticideUsedInHomePast7days', 'ChemicalWeedKillUsedExteriorHomePast7days',
                'Past7DaysCardio20Minutes+', 'Past7DaysStrengthTone20Minutes+', 'Past7DaysCardio60Minutes+']]

df.describe()







,ID,Gender,WeightAt25Yrs,HeightAt25Yrs,PlateletCount,CreatinePhosphokinase,HasArthritis,HasAsthma,AllergyCongestionPast12Months,Breast_fed,UseMilk>=5x/wk,DrankMilkAge13to17yrs,DrankMilkAge18to35yrs,ChemicalPesticideUsedInHomePast7days,ChemicalWeedKillUsedExteriorHomePast7days,Past7DaysCardio20Minutes+,Past7DaysStrengthTone20Minutes+,Past7DaysCardio60Minutes+
count,0.0,9813.000000,4842.000000,2667.000000,8544.000000,6542.000000,0.0,9422.000000,3680.000000,0.0,5561.000000,4126.000000,4126.000000,7532.000000,7531.000000,655.000000,655.000000,2887.000000
mean,NaN,1.509426,567.920074,373.831646,251.195108,153.986090,NaN,1.849395,1.716033,NaN,1.956662,2.663354,2.341978,1.925783,2.061214,3.413740,2.172519,5.805681
std,NaN,0.499937,1975.492188,1716.831150,66.054022,185.262311,NaN,0.422699,0.556719,NaN,0.875584,0.828785,0.898817,0.533254,0.925856,4.320649,2.038432,5.354061
min,NaN,1.000000,55.000000,50.000000,18.000000,6.000000,NaN,1.000000,1.000000,NaN,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,NaN,1.000000,125.000000,63.000000,206.000000,75.000000,NaN,2.000000,1.000000,NaN,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,4.000000
50%,NaN,2.000000,150.000000,66.000000,244.000000,108.000000,NaN,2.000000,2.000000,NaN,2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,2.000000,7.000000
75%,NaN,2.000000,180.000000,70.000000,288.000000,173.000000,NaN,2.000000,2.000000,NaN,3.000000,3.000000,3.000000,2.000000,2.000000,5.000000,3.000000,7.000000
max,NaN,2.000000,9999.000000,9999.000000,723.000000,3966.000000,NaN,9.000000,9.000000,NaN,9.000000,9.000000,9.000000,9.000000,9.000000,99.000000,7.000000,99.000000


**Feature Generation**
new column should be created for BMI.  
#The formula is BMI = kg/m2 where kg is a person’s weight in kilograms and m2 is their height in meters squared.
'WeightAt25Yrs' in kg divided by the 'HeightAt25Yrs' in meters squared.  

In [29]:
df.insert(4, 'BMI', ['WeightAt25Yrs' / ('HeightAt25Yrs'**2), True])
#need to check the units for weight and height.
#https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/WHQ_H.htm#Component_Description
#English Instructions:
    #ENTER HEIGHT IN FEET AND INCHES OR METERS AND CENTIMETERS.
    #RECORD CURRENT WEIGHT. ENTER WEIGHT IN POUNDS OR KILOGRAMS. 
    #IF PREGNANT, ASK FOR WEIGHT BEFORE PREGNANCY. 

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

In [6]:
#https://wwwn.cdc.gov/Nchs/Nhanes/Search/variablelist.aspx?Component=Questionnaire&CycleBeginYear=2013
##The variable dictionary to understand the code at the top of each column.  

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10175 entries, 0 to 10174
Columns: 952 entries, ACD011A to WHQ520
dtypes: float64(944), int64(6), object(2)
memory usage: 73.9+ MB


Medical Conditions  MCQ084 
difficulties in thinking or remembering that can make a big difference in everyday activities.
The next question asks about difficulties in thinking or remembering that can make a big difference in everyday activities. This does not refer to occasionally forgetting your keys or the name of someone you recently met. This refers to things like confusion or memory loss that are happening more often or getting worse. We want to know how these difficulties impact {you/SP}. During the past 12 months, {have you/has she/has he} experienced confusion or memory loss that is happening more often or is getting worse?

In [8]:
data.MCQ084.value_counts()

2.0    1521
1.0     318
9.0       1
Name: MCQ084, dtype: int64

In [9]:
data.MCQ084.count()

1840

https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/MCQ_H.htm#MCQ084
- 1521 is the count of No
- 318 is the count of Yes
- 1 is the count of don't know
- 7,930 are missing

In [10]:
data.MCQ084.tail(10)

10165    NaN
10166    2.0
10167    1.0
10168    NaN
10169    NaN
10170    NaN
10171    NaN
10172    NaN
10173    NaN
10174    NaN
Name: MCQ084, dtype: float64

In [11]:
#drop the NaN values.  How many rows re
memory_difficulty= data.MCQ084.dropna(axis = 0)
memory_difficulty.tail(10)

#axis=0  row   axis = 1  column

10127    2.0
10130    2.0
10135    2.0
10145    2.0
10151    2.0
10155    2.0
10160    2.0
10161    1.0
10166    2.0
10167    1.0
Name: MCQ084, dtype: float64

In [12]:
memory_difficulty.count()

1840

https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/PAQ_H.htm
-Past wk # days cardiovascular exercise  Physical Activity PAQ677
the first column is the number od days, the second column is the count
99 represents "I don't know"
-8805 are missing

Physical Activity PAQ677 On how many of the past 7 days did {you/SP} exercise or participate in physical activity for at least 20 minutes that made {you/him/her} sweat and breathe hard, such as basketball, soccer, running, swimming laps, fast bicycling, fast dancing, or similar activities?

In [13]:
data.PAQ677.value_counts()

5.0     154
0.0     111
3.0     110
2.0      81
4.0      65
1.0      62
7.0      52
6.0      43
99.0      1
Name: PAQ677, dtype: int64

In [14]:
data.PAQ677.count()

679

-Past wk # days strengthened muscles Physical Activity PAQ678
the first column is the number od days, the second column is the count
-8805 are missing

Physical Activity PAQ678 On how many of the past 7 days did {you/SP} do exercises to strengthen or tone {your/his/her} muscles, such as push-ups, sit-ups, or weight lifting?

In [15]:
data.PAQ678.value_counts()

0.0    207
2.0    109
3.0    106
1.0     96
5.0     77
4.0     39
7.0     32
6.0     13
Name: PAQ678, dtype: int64

In [16]:
data.PAQ678.count()

679

-Days physically active at least 60 min.
-6495 are missing
first column is number of days.
99 represents the don't know, which in this set was 8.

Physical Activity PAQ706 During the past 7 days, on how many days {were you/was SP} physically active for a total of at least 60 minutes per day? Add up all the time {you/he/she} spent in any kind of physical activity that increased {your/his/her} heart rate and made {you/him/her} breathe hard some of the time.

In [17]:
data.PAQ706.value_counts()

7.0     1799
5.0      314
3.0      194
4.0      183
2.0      159
0.0      131
6.0      111
1.0       90
99.0       8
Name: PAQ706, dtype: int64

In [18]:
data.PAQ706.count()

2989